## Подготовка данных для занятия

In [1]:
import re

import pandas as pd
import pymorphy2


text_df = pd.read_csv("/home/arcry/anaconda3/SkillBox/files/Strings/content_description.csv", sep='\t')
text_df.head()

# разбиваем тест на слова
corpus = []
# регулярка для поиска слов
regular_expr = r'\w+'
reg_expr_compiled = re.compile(regular_expr)
# формируем датасет из отдельных слов
for raw_text in text_df.description.values:
    # приводим к нижнему регистру
    raw_text_lower = raw_text.lower()
    # разбиваем текст на слова
    text_by_words = reg_expr_compiled.findall(raw_text_lower) 
    corpus.append(text_by_words)

# нормализация текста
normalized_corpus = []
morph = pymorphy2.MorphAnalyzer()
# нормализуем каждое слово в тексте
for token_list in corpus:
    normalized_token_list = []
    for word in token_list:
        parsed_token = morph.parse(word)
        normal_form = parsed_token[0].normal_form
        normalized_token_list.append(normal_form)
    normalized_corpus.append(normalized_token_list)

# превращаем в DataFrame
doc_count = len(normalized_corpus)
doc_ids = []
tokens = []
# формируем два списка-колонки датафрейма
for doc_id in range(doc_count):
    for token in normalized_corpus[doc_id]:
        doc_ids.append(doc_id)
        tokens.append(token)

tokens_df = pd.DataFrame({
    'doc_id': doc_ids,
    'word': tokens
})
# дамми-столбец
tokens_df = tokens_df.assign(dummy = 1)
# аггрегируем статистики
word_count_df = tokens_df.groupby(['doc_id','word'])['dummy'].count().reset_index()

word_count_df[word_count_df.doc_id==0].sort_values(by='dummy', ascending=False).head(10)

,doc_id,word,dummy
6,0,в,10
29,0,и,7
37,0,который,5
111,0,шерлок,4
77,0,с,4
84,0,сериал,4
75,0,риколетти,3
31,0,история,2
21,0,для,2
49,0,о,2


# 10.3  Домашняя работа

10.3.1 **Задание простого уровня** Для каждого слова подсчитайте процент документов, в которых содержится это слово. Сформируйте dataframe doc_frequency. Процен документов вычисляется по формуле
$$
d = \frac{m}{n} \times 100
$$
где $m$ - количество документов, в которых встретилось это слово, а $n$ - общее количество документов. Значение в процентах округлите до целых.

Посмотрите какие слова оказались в топе - это предлоги и частицы. Такие слова присутствуют во всех документах, а значит, их можно удалить из текста, чтобы оставшиеся слова были более "осмысленными". Это упростит анализ текста.

In [10]:
def words(word, morph=pymorphy2.MorphAnalyzer()): # исключим все предлоги, частицы, союзы, междометия, местоимения
    if morph.parse(word)[0].tag.POS not in ['INTJ', 'PRCL', 'CONJ', 'PREP', 'NPRO', 'PRED']:
        return True
    else:
        return False
    

In [11]:
word_count_df_filt = word_count_df[word_count_df['word'].apply(words)]
word_count_df_filt.head(10)

,doc_id,word,dummy
0,0,1895,1
2,0,абсолютно,1
3,0,англия,1
4,0,безобразный,1
5,0,быть,2
7,0,ватсон,1
8,0,весь,2
9,0,викторианский,1
10,0,внимательный,1
11,0,впервые,1


In [12]:
# сбросим счетчик dummy таким образом, чтобы при нахождении слова в документе dummy = 1, а не количеству нахождений в документе
word_count_df_filt.loc[:, 'dummy'] = 1

/home/arcry/anaconda3/envs/myenv/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [13]:
# определим количество слов в каждом документе
word_count_df_word = word_count_df_filt.groupby('word').count().sort_values('dummy').reset_index()[['word', 'dummy']]
word_count_df_word

,word,dummy
0,1895,1
1,путь,1
2,проходить,1
3,проходимец,1
4,просмотр,1
...,...,...
586,тот,4
587,смотреть,5
588,год,5
589,онлайн,6


In [14]:
# определим частоту появлений слов в документах взяв за количество документов len(word_count_df_filt['doc_id'].unique()) = 8
doc_frequency = word_count_df_word[['word', 'dummy']]
doc_frequency ['dummy_perc'] = doc_frequency['dummy'].apply(lambda x: x / len(word_count_df_filt['doc_id'].unique()) * 100)
doc_frequency

,word,dummy,dummy_perc
0,1895,1,12.5
1,путь,1,12.5
2,проходить,1,12.5
3,проходимец,1,12.5
4,просмотр,1,12.5
...,...,...,...
586,тот,4,50.0
587,смотреть,5,62.5
588,год,5,62.5
589,онлайн,6,75.0


10.3.2 **Задание среднего уровня**

Воспользуйтесь регулярными выражениями, чтобы извлечь из текста все пары **имя+фамилия**.

* неформальное описание регулярки: пара слов идущая друг за другом, каждое из которых начинается с заглавной буквы
* анализировать нужно только `doc_id=3`
* текст берём из исходного датафрейма `text_df`
* заглавная буква в русскоязычном тексте соответствует символьному классу `r'[A-Я]*'`

In [15]:
raw_text = text_df.description.values[3]
# регулярка - её нужно поправить
reg_expr = '( [А-Я]\w+ [А-Я]\w+)'
# компилируем регулярное выражение
reg_expr_compiled = re.compile(reg_expr)
# применяем выражение к тексту
for g in reg_expr_compiled.findall(raw_text):
    print(g)
# невозможно извлечь только имя+фамилию, потому что в тексте могут быть слова
# с заглавных букв в начале предложения, за которыми идет имя

 Стивена Фрая
 Джоном Дженксом
 Теда Уоллеса
 Теперь Тед
 Вскоре Тед


In [174]:
raw_text

'Экранизация сатирического бестселлера Стивена Фрая, мастерски воплощенная британским режиссером и продюсером Джоном Дженксом.   У Теда Уоллеса было славное прошлое, и вот-вот его книги должны были пополнить стройный ряд томов британских классиков. Но его жуткий характер вкупе с алкоголизмом и мизантропией в запущенной стадии испортили все. Теперь Тед зарабатывает на жизнь тем, что поносит в своих статьях спектакли, которые вообще-то не стоят его внимания. И продолжает пить. Вскоре Тед лишается и этого: его бесконечное ворчание достало всю редакцию.  Готовясь к голодной и бесславной смерти, внезапно Уоллес получает выгодное предложение от своей крестницы Джейн. Девушка недавно излечилась от болезни благодаря чуду и теперь предлагает Теду погостить в аристократическом доме, чтобы осветить происходящие здесь чудеса исцеления.  Старый скептик соглашается из корыстных побуждений и даже представить не может, сколько сюрпризов ему уготовано.  Смотреть онлайн эту комедию о том, что даже самые

10.3.3 **Задание сложного уровня**. Словарь `genre_dict` содержит слова, которые являются характерными для того или иного жанра. Пользуясь словарём, а так же таблицей `word_count_df`, сформируйте таблицу двумя колонками `doc_id | genre` с жанрами фильмов.

Для этого нужно
* превратить словарь `genre_dict` в DataFrame формата `word | genre`
* соединить полученный датафрейм с помощью функции `merge` c ранее полученным датафреймом `word_count_df`, котороый содержит распределение слов по документам. Воспользуйтесь методом соединения `inner`
* для каждого документа выбрать жанр документа - это совокупность жанров слов отдельных слов. У одного контента может быть несколько жанров

In [16]:
import itertools
import pandas as pd

genre_dict = {
    'комедия': ['сатирический', 'авантюрный', 'забавный'],
    'мелодрама': ['выбор', 'позор'],
    'сказка': ['приключения', 'милый', 'семейный'],
    'детектив': ['тайна', 'разгадать', 'загадочный'], 
    'триллер': ['ужас', 'зловещий', 'нерв']
}

nested_genres =[[(i, j) for j in genre_dict[i]] for i in genre_dict]
# переводим из словаря к более удобному виду
flatten_genres = list(itertools.chain(*nested_genres))
# создаём DataFrame
genres_df = pd.DataFrame(flatten_genres, columns = ['genre', 'character_word'])

In [50]:
doc_vs_genres = word_count_df.merge(genres_df, how='inner', left_on='word', right_on='character_word') #объединим две таблицы
doc_vs_genres.sort_values('doc_id')

,doc_id,word,dummy,genre,character_word
0,0,загадочный,1,детектив,загадочный
1,0,разгадать,1,детектив,разгадать
2,0,тайна,1,детектив,тайна
3,0,ужас,1,триллер,ужас
5,1,милый,1,сказка,милый
6,2,выбор,1,мелодрама,выбор
7,2,позор,1,мелодрама,позор
8,3,сатирический,1,комедия,сатирический
9,4,авантюрный,1,комедия,авантюрный
10,4,забавный,1,комедия,забавный


In [49]:
doc_to_genres = pd.DataFrame(doc_vs_genres.groupby('doc_id')['genre'].unique()) # выделим жанры каждого документа
doc_to_genres

,genre
doc_id,
0,"[детектив, триллер]"
1,[сказка]
2,[мелодрама]
3,[комедия]
4,[комедия]
6,[триллер]
